In [ ]:
!pip install beautifulsoup4 requests pdfkit pandas

MAIN CODE WIKIPEDIA

In [ ]:
import requests
from bs4 import BeautifulSoup
import re


def scrape_wikipedia_page(url):
    try:

        response = requests.get(url, verify=False)
        response.raise_for_status()


        soup = BeautifulSoup(response.content, "html.parser")


        title_tag = soup.find('h1')
        if title_tag:
            title = title_tag.get_text(strip=True)
        else:
            title = "Title not found"


        sections = {}
        current_section = title
        sections[current_section] = []


        for tag in soup.find_all(['h2', 'h3', 'h4', 'p', 'ul', 'ol', 'table']):
            if tag.name in ['h2', 'h3', 'h4']:
                current_section = tag.get_text(strip=True)
                sections[current_section] = []



            elif tag.name == 'p':
                paragraph_text = ''

                for content in tag.contents:
                    if isinstance(content, str):
                      paragraph_text += content.strip()
                    elif content.name == 'a':
                      paragraph_text += ' ' + content.get_text(strip=True) + ' '

                # Normalize spaces and add the paragraph text to the section
                sections[current_section].append(re.sub(r'\s+', ' ', paragraph_text).strip())


            elif tag.name in ['ul', 'ol']:  # Lists (unordered and ordered)
                list_items = [li.get_text(strip=True) for li in tag.find_all('li')]
                sections[current_section].append("\n".join(f"• {item}" for item in list_items))
            elif tag.name == 'table':  # Tables
                table_data = []
                rows = tag.find_all('tr')
                for row in rows:
                    cells = row.find_all(['th', 'td'])
                    row_text = [cell.get_text(strip=True) for cell in cells]
                    table_data.append(" | ".join(row_text))  # Join cells with a pipe separator for readability
                sections[current_section].append("\n".join(table_data))

        return title, sections
    except requests.RequestException as e:
        print(f"Error scraping {url}: {e}")
        return None, None

# Function to save scraped data to a text file
def save_scraped_data_to_file(title, sections, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        # Save the title of the page at the top
        file.write(f"Title: {title}\n")
        file.write("=" * len(title) + "\n\n")

        # Save each section with its heading and content (including lists and tables)
        for heading, content in sections.items():
            file.write(f"{heading}\n")
            file.write("-" * len(heading) + "\n")  # Underline the heading
            for item in content:
                file.write(f"{item}\n\n")

# Example usage
if __name__ == "__main__":

    wiki_url = "https://pittsburghpa.gov/mayor/index.html"


    output_file = "CityHall.txt"


    title, sections = scrape_wikipedia_page(wiki_url)


    if title and sections:

        save_scraped_data_to_file(title, sections, output_file)
        print(f"Scraped data saved to {output_file}")
    else:
        print("Failed to scrape the Wikipedia page.")


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pittsburghpa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Scraped data saved to CityHall.txt


Scrape Links in Text File

In [ ]:
def is_valid_url(url):

    return url.startswith("http://") or url.startswith("https://")


def scrape_content(link):
    cleaned_link = link.strip().rstrip("\\/")

    if not is_valid_url(cleaned_link):
        print(f"Skipping invalid URL: {cleaned_link}")
        return

    try:
        response = requests.get(cleaned_link)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        headings = [heading.get_text().strip() for heading in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]
        paragraphs = [para.get_text().strip() for para in soup.find_all('p')]

        with open("scraped_content.txt", "a", encoding="utf8") as out_file:
            out_file.write(f"URL: {cleaned_link}\n")
            out_file.write("Headings:\n")
            for heading in headings:
                out_file.write(f"{heading}\n")
            out_file.write("\nParagraphs:\n")
            for para in paragraphs:
                out_file.write(f"{para}\n")
            out_file.write("\n" + "="*80 + "\n")

    except requests.exceptions.RequestException as e:
        print(f"Error scraping {cleaned_link}: {e}")


with open("/content/Links.rtf", "r") as file:
    for link in file.readlines():
        scrape_content(link)


Skipping invalid URL: {\rtf1\ansi\ansicpg1252\cocoartf2761
Skipping invalid URL: \cocoatextscaling0\cocoaplatform0{\fonttbl\f0\fswiss\fcharset0 Helvetica;}
Skipping invalid URL: {\colortbl;\red255\green255\blue255;}
Skipping invalid URL: {\*\expandedcolortbl;;}
Skipping invalid URL: \margl1440\margr1440\vieww11520\viewh8400\viewkind0
Skipping invalid URL: \pard\tx720\tx1440\tx2160\tx2880\tx3600\tx4320\tx5040\tx5760\tx6480\tx7200\tx7920\tx8640\pardirnatural\partightenfactor0
Skipping invalid URL: 
Skipping invalid URL: \f0\fs24 \cf0 https://www.visitpittsburgh.com/blog/pittsburgh-vegan-restaurants
Error scraping https://www.sportspittsburgh.com/about-sportspittsburgh/}: 404 Client Error: Not Found for url: https://www.sportspittsburgh.com/about-sportspittsburgh/%7D/


In [ ]:
import warnings
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re

from urllib3.exceptions import InsecureRequestWarning
warnings.simplefilter('ignore', InsecureRequestWarning)


def clean_text(text):

    text = re.sub(r'[^\w\s.,\'"!?]', ' ', text)

    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)


    text = re.sub(r'\s+', ' ', text).strip()

    return text


def scrape_page(url, content_file):
    try:
        response = requests.get(url, verify=False)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')


            title = soup.find('title').get_text(" ", strip=True) if soup.find('title') else 'No Title'
            content_file.write(f"Page URL: {url}\n")
            content_file.write(f"Title: {clean_text(title)}\n\n")


            paragraphs = soup.find_all('p')
            for paragraph in paragraphs:
                cleaned_paragraph = clean_text(paragraph.get_text(" ", strip=True))
                content_file.write(cleaned_paragraph + '\n')


            lists = soup.find_all(['ul', 'ol'])
            for lst in lists:
                list_items = lst.find_all('li')
                for item in list_items:
                    cleaned_item = clean_text(item.get_text(" ", strip=True))
                    content_file.write(f"- {cleaned_item}\n")


            tables = soup.find_all('table')
            for table in tables:
                rows = table.find_all('tr')
                for row in rows:
                    columns = row.find_all(['th', 'td'])
                    cleaned_row = ' | '.join([clean_text(col.get_text(" ", strip=True)) for col in columns])
                    content_file.write(cleaned_row + '\n')

            content_file.write("\n--- End of page content ---\n\n")
        else:
            print(f"Failed to scrape {url}")
    except Exception as e:
        print(f"Error scraping {url}: {e}")


def find_subpages(main_url):
    try:
        response = requests.get(main_url, verify=False)
        subpage_links = []

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')


            for link in soup.find_all('a', href=True):
                href = link['href']

                full_url = urljoin(main_url, href)


                if main_url in full_url:
                    subpage_links.append(full_url)

        return subpage_links
    except Exception as e:
        print(f"Error finding subpages: {e}")
        return []

def scrape_website_with_subpages(main_url, content_file, subpages_file):
    with open(content_file, 'w', encoding='utf-8') as content, open(subpages_file, 'w', encoding='utf-8') as subpages:
        scrape_page(main_url, content)
        subpage_links = find_subpages(main_url)

        print(f"Found {len(subpage_links)} subpages to scrape.\n")
        content.write(f"Found {len(subpage_links)} subpages to scrape.\n\n")
        subpages.write(f"Subpages scraped from {main_url}:\n\n")

        for subpage in subpage_links:
            subpages.write(subpage + '\n')
            scrape_page(subpage, content)


main_url = "https://pittsburghpa.gov/mayor/city-staff"
content_file = "scraped_content.txt"
subpages_file = "scraped_subpages.txt"
scrape_website_with_subpages(main_url, content_file, subpages_file)


Found 12 subpages to scrape.

